In [ ]:
pip install mediacloud

In [ ]:
pip install newspaper3k

In [ ]:
def limited_matching_stories(mc_client, q, fq):
    """
    Return all the stories matching a query within Media Cloud. Page through the results automatically.
    :param mc_client: a `mediacloud.api.MediaCloud` object instantiated with your API key already
    :param q: your query
    :param fq: your date range query
    :return: a list of media cloud story items
    """
    last_id = 0
    more_stories = True
    stories = []
    while (len(stories) < 500 and more_stories):
        page = mc_client.storyList(q, fq, last_processed_stories_id=last_id, rows=500, sort='processed_stories_id')
        print("  got one page with {} stories".format(len(page)))
        if len(page) == 0:
            more_stories = False
        else:
            stories += page
            last_id = page[-1]['processed_stories_id']
    return stories

In [ ]:
def removeQuotes(words):
  newString = ""
  inQuote = False
  for word in words:
    if (word[0] == '"'):
      inQuote = True
    if (not inQuote):
      newString += (word + " ")
    if (word[len(word) - 1] == '"'):
      inQuote = False
    
  return newString
    

In [ ]:
import requests

def url_parcer(url):
  try:
    data = []
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    words = article.text.split()
    print(words)
    wordCount = len(words)
    topicAppearances = words.count(topic) 
    numQuotes = 0
    for word in words:
      if (word[0] == '"'):
        numQuotes += 1
    r = requests.post("https://api.deepai.org/api/sentiment-analysis", data = {'text': removeQuotes(words)}, headers = {'api-key': '4a102c57-b6c1-4f44-b192-294b47e8cda2'})
    data.append(wordCount)
    data.append(topicAppearances)
    data.append(numQuotes)
    data.append(r.json()['output'])
    return words
  except:
    return []

In [ ]:
import mediacloud.api
import datetime
import csv
import pandas

mc = mediacloud.api.MediaCloud('74ebee12e7edefed3f3e684af66bf346e319bc117668b02ceafeca2cfc554bfd')
startDate = datetime.date(2020, 10, 15)
endDate = datetime.date(2020, 10, 17)
date_range_2019 = mc.dates_as_query_clause(startDate, endDate)
topic = input("Enter a topic: ")
uk_query = '"' + topic + '" and tags_id_stories:8878466'
#print(uk_query)
res = mc.storyCount(uk_query, date_range_2019)
all_stories = limited_matching_stories(mc, uk_query, date_range_2019)
#print(all_stories)
#print(res)#['count'])

import mediacloud.tags
for s in all_stories:
    # see the "language" notebook for more details on themes
    theme_tag_names = ','.join([t['tag'] for t in s['story_tags'] if t['tag_sets_id'] == mediacloud.tags.TAG_SET_NYT_THEMES])
    s['themes'] = theme_tag_names
# now write the CSV
#import csv
fieldnames = ['stories_id', 'publish_date', 'title', 'url', 'language', 'ap_syndicated', 'themes', 'media_id', 'media_name', 'media_url']
with open('story-list.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')
    writer.writeheader()
    for s in all_stories:
        writer.writerow(s)

"brexit" and tags_id_stories:8878466 , 662194 results

In [ ]:
import newspaper
from newspaper import Article
from newspaper import fulltext
import nltk
import requests
import csv
#nltk.download('punkt')

urls = []
dates = []

with open("story-list.csv") as csv_file:
  csv_reader = csv.reader(csv_file, delimiter = ',')
  for r in csv_reader:
    if r[3] not in urls: # Eliminates repeats
      urls.append(r[3])
      dates.append(r[1])
urls = urls[1:]
print(topic)
print(len(urls))
url_data = {}
for url in urls:
  url_data[url] = url_parcer(url)
print(url_data)
print("done")

#print(urls)

#a = Article(urls[1])

#a.download()
#a.parse()
#a.nlp()

#html = requests.get("https://www.dailystar.co.uk/news/latest-news/biker-mum-dies-suddenly-heart-22858155").text
#text = fulltext(html)
#print(text)

#print("Keywords:", a.keywords)
#print("Authors:", a.authors)

#print(a.title)
#print(a.text)

elmo
5
{'https://nationalinterest.org/blog/reboot/wwii-what-was-it-parachute-france-d-day-170697': [], 'https://www.dailystar.co.uk/news/latest-news/biker-mum-dies-suddenly-heart-22858155': [], 'https://muppet.fandom.com/wiki/Episode_4215?diff=1333212&oldid=1331667': [], 'https://muppet.fandom.com/wiki/Sandbox:Sesame_Street_Season_19_Segments?diff=1333398&oldid=1310238': [], 'https://muppet.fandom.com/wiki/Ludwig_van_Beethoven?diff=1333473&oldid=1301873': []}
done


In [ ]:
ultimateMeanVibe = 0
for data in url_data:
  vibeCount = 0
  print(url_data)
  vibeCount += url_data[data][3].count('Verynegative') * 1
  vibeCount += url_data[data][3].count('Negative') * 2
  vibeCount += url_data[data][3].count('Neutral') * 3
  vibeCount += url_data[data][3].count('Positive') * 4
  vibeCount += url_data[data][3].count('Verypositive') * 5
  ultimateMeanVibe += (vibeCount / len(url_data[data][3]))
  vibeCount /= len(url_data[data][3])
  vibeCount -= 3 # centers data about 0
  url_data[data].append(vibeCount)


ultimateMeanVibe /= len(url_data)
ultimateMeanVibe -= 3
penalties = {}
meanVibeWeight = 3 # weight to change over time
for data in url_data:
  penalties[data] = (abs(url_data[data][4] - ultimateMeanVibe)) * meanVibeWeight # data piece will be anywhere from 0 to 18 penalty, where it normally does not exceed 9

print(dates[0])


1
